# Understanding Household Energy Consumption: A Deep Learning Approach

Welcome to a Deep Learning tutorial created by Hertie School's Masters of Data Science for Public Policy students. In it you will be able to apply sequence modeling techniques to analyze and predict household electric power consumption. We used the "Individual Household Electric Power Consumption" dataset from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption), which includes diverse measures of electricity usage in a household over four years in Sceaux (7km of Paris, France) between December 2006 and November 2010 (47 months). This tutoral will focus on employing time series forecasting methods, like ARIMA, Recurrent Neural Networks (RNNs), and Long Short-Term Memory (LSTM) networks, to understand patterns in energy usage, forecast future consumption and comparing each other in terms of performance and results. 

## **Understand the Data**
First, we need to understand the data we are going to be working with. Install the necessary libraries:

In [1]:
# Based on https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/

# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import os

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xe

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xe

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xe

SystemError: initialization of _pywrap_checkpoint_reader raised unreported exception